In [0]:
#Tensorflow Import and GPU recognition
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13360194886293692580, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17105441835016608308
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15499978064239632271
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11150726272
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13760408617209912280
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip '/content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip' -d '/content'

Streaming output truncated to the last 5000 lines.
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_16_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_17_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_181_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_182_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_183_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_184_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_185_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_186_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_187_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_188_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_189_100.jpg  
  inflating: /co

In [0]:
#Importing Keras and other useful libs
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from glob import glob
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [0]:
#Defining dataset path and saving it for future usage
path = '/content/fruits-360_dataset/fruits-360'
os.listdir(path)

['test-multiple_fruits', 'papers', 'Test', 'Training', 'LICENSE', 'readme.md']

In [0]:
#Number of pictures in Training folder
training_files = glob(os.path.join(path,'Training', '*/*.jpg'))
image_num = len(training_files)
print("Number of Images: ",image_num)

Number of Images:  60498


In [0]:
#Number of pictures in Test Folder
testing_files = glob(os.path.join(path, 'Test', '*/*.jpg'))
img_num = len(testing_files)
print("Number of Images: ", img_num)

Number of Images:  20622


In [0]:
#Print category details
image_count = []
class_names = []
print('{:18s}'.format('Class'), end='')
print('Count:')
print('-'*24)
for folder in os.listdir(os.path.join(path,'Training')):
  folder_count = len(os.listdir(os.path.join(path,'Training',folder)))
  image_count.append(folder_count)
  class_names.append(folder)
  print('{:20s}'.format(folder), end='')
  print(folder_count)
print('-'*24)
print('Number of Classes:', len(class_names))
print('Average number of images per Class: ', np.array(image_count).mean())

Class             Count:
------------------------
Apple Braeburn      492
Rambutan            492
Orange              479
Potato Red          450
Cantaloupe 1        492
Beetroot            450
Mulberry            492
Apple Red 3         429
Onion Red Peeled    445
Pear Williams       490
Cactus fruit        490
Pear Forelle        702
Kohlrabi            471
Pear Red            666
Carambula           490
Tomato 4            479
Lychee              490
Pear Kaiser         300
Apple Red 2         492
Grape White         490
Pomelo Sweetie      450
Pepper Yellow       666
Potato Sweet        450
Tomato 1            738
Passion Fruit       490
Pear Abate          490
Apple Red Yellow 2  672
Grapefruit White    492
Grape Pink          492
Walnut              735
Tamarillo           490
Cocos               490
Peach Flat          492
Chestnut            450
Pitahaya Red        490
Apple Red 1         492
Pear                492
Pineapple Mini      493
Cherry 1            492
Grape White 4 

In [0]:
#Definitive paths
train_out_path = os.path.join(path,'Training')
test_out_path = os.path.join(path, 'Test')
print(train_out_path)
print(test_out_path)

/content/fruits-360_dataset/fruits-360/Training
/content/fruits-360_dataset/fruits-360/Test


In [0]:
#2 Approach: Inception v3 pre-trained
from keras_applications.vgg16 import VGG16
from keras_applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras import optimizers
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras import regularizers
from keras.layers.convolutional import *
from keras.models import Model

In [0]:
#Implementing some augmentation to avoid overfitting on the training generator
train_datagenerator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
train_and_val_generator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True)
test_datagenerator = ImageDataGenerator(rescale= 1./255)

In [0]:
#Creating Batches
image_size = (80, 80)
train_batches = train_datagenerator.flow_from_directory(train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='training', seed = 20052020)
val_batches = train_datagenerator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='validation', seed = 20052020)
train_val_batches = train_and_val_generator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, seed = 20052020)
test_batches = test_datagenerator.flow_from_directory(directory=test_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, shuffle=False)

Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Found 60498 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [0]:
inception_v3_model = applications.InceptionV3(include_top=False, weights='imagenet', input_shape=image_size+(3,))
#Using pretrained
for layer in inception_v3_model.layers:
  layer.trainable=False
#Adding Layers at the top
top = inception_v3_model.output
top = GlobalAveragePooling2D(data_format='channels_last', name='avg_pool')(top)


#Adding Classification layer with softmax activation function(needed for categorical predictions)
predictor = Dense(120 ,activation='softmax', name='predictor')(top)
inception_v3_model = Model(inputs=inception_v3_model.input, outputs=predictor)

In [0]:
inception_v3_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 80, 80, 3)    0                                            
__________________________________________________________________________________________________
conv2d_377 (Conv2D)             (None, 39, 39, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_377 (BatchN (None, 39, 39, 32)   96          conv2d_377[0][0]                 
__________________________________________________________________________________________________
activation_377 (Activation)     (None, 39, 39, 32)   0           batch_normalization_377[0][0]    
____________________________________________________________________________________________

In [0]:
#Compiling the model
inception_v3_model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
inception_v3_model.save_weights('Initial weights.h5')

In [0]:
#Training and Validating the model: Round 1
wait = 1
epochs = 15
model_name='Inception v3'
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=wait)

In [27]:
inception_v3_model.fit_generator(train_batches, epochs=5 , verbose=1, shuffle=True, validation_data=val_batches, callbacks=[early_stopping])
inception_v3_model.load_weights('Initial weights.h5')
trainingMdl = inception_v3_model.fit_generator(train_val_batches, epochs=early_stopping.stopped_epoch , verbose=1, shuffle=True, validation_data=val_batches)

Epoch 1/5
1514/1514 [==============================] - 187s 124ms/step - loss: 1.5425 - accuracy: 0.6159 - val_loss: 9.7848 - val_accuracy: 0.0865
Epoch 2/5
1514/1514 [==============================] - 180s 119ms/step - loss: 1.4442 - accuracy: 0.6363 - val_loss: 12.0777 - val_accuracy: 0.0771
Epoch 1/1
 424/1891 [=====>........................] - ETA: 2:22 - loss: 3.5535 - accuracy: 0.2632

KeyboardInterrupt: ignored

In [28]:
trainingMdl = inception_v3_model.fit_generator(train_val_batches, epochs=15 , verbose=1, shuffle=True, validation_data=val_batches)

Epoch 1/15
1891/1891 [==============================] - 223s 118ms/step - loss: 1.9554 - accuracy: 0.5493 - val_loss: 12.3544 - val_accuracy: 0.0775
Epoch 2/15
1891/1891 [==============================] - 221s 117ms/step - loss: 1.5327 - accuracy: 0.6204 - val_loss: 14.5194 - val_accuracy: 0.0957
Epoch 3/15
1891/1891 [==============================] - 221s 117ms/step - loss: 1.4210 - accuracy: 0.6403 - val_loss: 20.9752 - val_accuracy: 0.0878
Epoch 4/15
1891/1891 [==============================] - 228s 121ms/step - loss: 1.3627 - accuracy: 0.6529 - val_loss: 21.9138 - val_accuracy: 0.0870
Epoch 5/15
1891/1891 [==============================] - 231s 122ms/step - loss: 1.3343 - accuracy: 0.6564 - val_loss: 12.6290 - val_accuracy: 0.0928
Epoch 6/15
1891/1891 [==============================] - 230s 121ms/step - loss: 1.3114 - accuracy: 0.6650 - val_loss: 14.4307 - val_accuracy: 0.0955
Epoch 7/15
1891/1891 [==============================] - 230s 121ms/step - loss: 1.2906 - accuracy: 0.6675 

In [0]:
import sys
sys.path.append('/content/drive/My Drive/Progetto Advanced Machine Learning/')
from utils import plot_history
history_fig = plot_history(trainingMdl)
history_fig.show()